# Gemini 2.0 Flash basics (text and text+image)

This notebook shows how to call Google's Gemini 2.0 Flash model for:
- Text generation
- Multi-turn chat
- Text + image prompts

It also explains key generation parameters and how to obtain an API key.

### get your api key from google ai studio

1. Visit **Google AI Studio** at `https://ai.google.dev` and sign in with your Google account.  
2. Go to **Get API key** (or **API keys**) in the left navigation.  
3. Click **Create API key**, choose the project if prompted, and copy the key.  
4. (Optional but recommended) Restrict the key to your environment and rotate it periodically.  
5. Store it securely, e.g., in an environment variable `GEMINI_API_KEY`.

> Note: The UI labels may change slightly over time. If something looks different, search for "API keys" within AI Studio.

### install dependencies

```bash
pip install --upgrade google-generativeai pillow
```

In [1]:
# configure the client
import os
import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv()

# Prefer environment variable; fall back to an input prompt (not echoed)
api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

# Create a model handle. If you get a "model not found" error,
# upgrade the SDK and confirm the model name you have access to.
model_name = "gemini-2.0-flash"
model = genai.GenerativeModel(
    model_name,
    system_instruction="You are a concise, helpful assistant."
)
print("Client configured. Model:", model_name)

Client configured. Model: gemini-2.0-flash


/Users/bhavishya/VSC Projects/Sessions/HoE/First GenAI App/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### generation parameters you will use

- **temperature**: randomness; higher = more creative, lower = more deterministic.  
- **top_p**: nucleus sampling; consider tokens from the smallest set whose cumulative probability ≥ *p*.  
- **top_k**: sample only from the top-*k* most probable tokens at each step.  
- **max_output_tokens**: hard limit on the length of the model's output.  
- **response_mime_type**: preferred format, e.g., `"text/plain"` or `"application/json"`.  
- **safety_settings**: optional policy filters for categories and thresholds.

Use these with `generation_config={...}` on each request.

In [7]:
# simple text generation with parameters
generation_config = {
    "temperature": 0.7,
    "top_p": 0.9,
    "top_k": 40,
    "max_output_tokens": 100,
    "response_mime_type": "text/plain",
}

prompt = "In 5 bullet points, explain the benefits of writing concise technical documentation."
resp = model.generate_content(prompt, generation_config=generation_config)
print(resp.text)

*   **Improved Clarity:** Concise documentation reduces ambiguity, making it easier for users to understand complex information.
*   **Increased Efficiency:** Shorter documentation saves time for both writers and readers, accelerating the learning and implementation process.
*   **Reduced Errors:** Clear and direct instructions minimize the chance of misinterpretation and subsequent errors.
*   **Enhanced Maintainability:** Concise documents are easier to update and maintain, ensuring accuracy and relevance over time.
*   **Better User Experience:** Respect


In [8]:
# multi-turn chat
chat = model.start_chat(history=[
    {"role": "user", "parts": "Hello!"},
    {"role": "model", "parts": "Hi there! How can I help today?"},
])

r1 = chat.send_message("Give me two fun facts about honeybees.")
print("Assistant:", r1.text, "\n")

r2 = chat.send_message("Now summarize that in one sentence.")
print("Assistant:", r2.text)

Assistant: Okay, here are two fun facts about honeybees:

1.  **Honeybees communicate through dance:** They perform a "waggle dance" to tell other bees the direction and distance of food sources.
2.  **Honeybees can recognize human faces:** Studies have shown they can distinguish between different human faces, which is impressive for such a small brain!
 

Assistant: Honeybees communicate through a waggle dance and can recognize human faces.



### text + image prompt

You can pass images along with text. Provide each image as a dict with `mime_type` and raw `data` bytes.

In [9]:
# text + image example
from pathlib import Path

# Replace this path with a local image path on your machine.
image_path = Path("images.jpeg")

# Load image bytes if available
parts = []
if image_path.exists():
    img_bytes = image_path.read_bytes()
    parts.append({
        "mime_type": "image/jpeg" if image_path.suffix.lower() in [".jpg", ".jpeg"] else "image/png",
        "data": img_bytes,
    })
else:
    print("Note: sample image not found. The request will be text-only.")

parts.append("Who do you see in this image?")
resp = model.generate_content(parts, generation_config={
    "temperature": 0.4,
    "max_output_tokens": 200,
    "response_mime_type": "text/plain",
})
print(resp.text)

Based on the image, I see the Mona Lisa, a famous portrait painted by Leonardo da Vinci.



### troubleshooting tips

- If you see `PermissionDenied` or `NotFound`, verify you have access to the model name and that your SDK is up to date.  
- Ensure your `GEMINI_API_KEY` is set and valid.  
- For large images, downscale before sending to reduce latency.  
- Handle errors with try/except and show helpful messages.